# KAN + MNIST + NTK
Purpose: Fit a KAN to the MNIST dataset, for benchmarking the KAN performance.

Furthermore, the PyTorch Lightning library is used for convenience.

In [1]:
# Imports
import numpy as np
import pandas as pd
import torch
from torch import nn
import lightning as L
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from lightning.pytorch.loggers import CSVLogger

import sys
sys.path.append('../convkans/kan_convolutional')
from KANLinear import *

# Setup Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Setup Randomness -- https://lightning.ai/docs/pytorch/stable/common/trainer.html
L.seed_everything(42, workers=True)

# CUDA Efficiency
torch.set_float32_matmul_precision('high')

Seed set to 42


In [2]:
# Dataset Setup
train_dataset = MNIST("./temp/", train=True, download=True)
test_dataset = MNIST("./temp/", train=False, download=True)

class LCDataset(Dataset): # Lightning Compatible Dataset
    def __init__(self, dataset, limit=-1):
        self.limit = limit
        if self.limit != -1:
            self.sub = list(np.random.permutation(np.arange(len(train_dataset)))[0:self.limit]) # Take a random sample of the first some elements.
            self.data = dataset.data[self.sub, :, :].view(-1, 28*28).type(torch.float32)
            self.target = torch.tensor((pd.get_dummies(pd.Series(dataset.targets[self.sub].numpy())).map(lambda x: 1 if x == True else 0)).values).type(torch.float32)
        else:
            self.data = dataset.data.view(-1, 28*28).type(torch.float32)
            self.target = torch.tensor((pd.get_dummies(pd.Series(dataset.targets.numpy())).map(lambda x: 1 if x == True else 0)).values).type(torch.float32)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]

train_loader = DataLoader(LCDataset(train_dataset, limit=500), batch_size=8, shuffle=True, num_workers=10)
test_loader = DataLoader(LCDataset(test_dataset), batch_size=8, num_workers=10)

In [3]:
# Model Declaration
class ClassicKAN(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            KAN(layers_hidden=[28*28, 240, 60, 10], grid_size=2, spline_order=2)
        )
    
    def forward(self, x):
        return self.net(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred, y)
        v1 = torch.argmax(y_pred, dim=1)
        v2 = torch.argmax(y, dim=1)
        accuracy = torch.sum(torch.eq(v1, v2)) / len(y)
        self.log("test loss (cross entropy)", loss)
        self.log("accuracy", accuracy)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        return self(batch)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        return optimizer

In [4]:
# Train + Test + Results
model = ClassicKAN()
trained_model = L.Trainer(max_epochs=100, deterministic=True, logger=CSVLogger("logs", name="MNISTKAN"))
trained_model.fit(model, train_loader)
trained_model.test(model, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params | Mode 
--------------------------------------------
0 | net  | Sequential | 1.2 M  | train
--------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.876     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Training: |                                               | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |                                                | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │     0.829800009727478     │
│ test loss (cross entropy) │    0.8854795098304749     │
└───────────────────────────┴───────────────────────────┘

[{'test loss (cross entropy)': 0.8854795098304749,
  'accuracy': 0.829800009727478}]

In [5]:
# Apply NTK
%run -i 'misc/introduction_code_modded_KAN_NTK.py'

def cross_entropy_loss_batch(y_hat, y):
    return F.cross_entropy(y_hat, y, reduction='none')

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
ntk_model = GaussianFit(model=model, device=device, noise_var=0.0)
ntk_model.fit(train_loader, optimizer, MSELoss_batch) # MSELoss seems to work better for NTK Models

In [6]:
def check_ntk_acc(model, dataloader):
    res = 0.0
    sumlength = 0
    model.eval()
    model.to(device)
    for it in iter(dataloader):
        x, y = it
        x = x.to(device)
        y = y.to(device)
        sumlength += len(x)
        res += (torch.argmax(model.forward(x), dim=1) == torch.argmax(y, dim=1)).sum()
    model.train()
    return res / sumlength

In [7]:
print(f'NTK Accuracy: {check_ntk_acc(ntk_model, test_loader)}')

NTK Accuracy: 0.28999999165534973


In [8]:
# R: True Value, C: Predicted Value
def make_predict_matrix(model, dataloader):
    res = np.zeros(shape=(10, 10), dtype=int)
    for it in iter(dataloader):
        x, y = it
        x = x.to(device)
        y = y.to(device)
        x_arg = torch.argmax(model.forward(x), dim=1)
        y_arg = torch.argmax(y, dim=1)
        for i in range(len(x_arg)):
            res[y_arg[i], x_arg[i]] += 1
    return res

In [9]:
print(make_predict_matrix(ntk_model, test_loader))

[[314 101  55  58  51  89  79  96  96  41]
 [ 39 428  49 104 170  15 133  68  66  63]
 [ 48  81 326  91 104  56  98 104  72  52]
 [ 67  68 114 344  62  44  79 100  63  69]
 [ 74  79  79  74 268  61  87  82  63 115]
 [ 73  62  50  89  49 215  77  69 116  92]
 [ 76 114  73  41  94  46 268  35  77 134]
 [127  62  47 136  63  37  95 308  63  90]
 [ 90  72  64  85  59  73  79  75 259 118]
 [ 49  64  95 130 161  71 124  90  55 170]]


In [10]:
print(make_predict_matrix(model, test_loader))

[[ 874    1   24    3    2   40   29    0    5    2]
 [   2 1109    4    1    0    3    3    3   10    0]
 [  29   12  822   34   31   10   26   18   47    3]
 [  17    7   29  804   11   49   15   21   40   17]
 [   1    0    6    2  839    0   17    4   20   93]
 [  23   11   17   71   15  636   33   12   63   11]
 [  59    2   24    2   24   27  816    0    4    0]
 [   2   15   12   12   24    8    3  882   18   52]
 [  48   22    6   32   15   47   23   29  720   32]
 [  25    6    4    5   88    3    2   70   10  796]]
